In [1]:
import pandas as pd

# Для планировщика

вопросы

In [2]:
tasks = ["Покажи лучший офис по обслуживанию клиентов",
         "Проанализируй офис n на предмет динамики клиентопотока за апрель 2025 года и предложи актуальный режим работы офиса",
         "Определи причину длительного ожидания в офисе n",
         "Сформируй отчёт за апрель по талонам для офиса n",
         "Как можно оптимизировать работу офиса n чтобы сократить время ожидания клиентов"]

ответы

In [3]:
instructions = ["""Найти в таблице Coupons какой офис имеет наименьшее среднее значение по времени (минуты) ожидания клиентов
                Найти в таблице Coupons какой офис имеет наименьшее среднее значение по скорости (минуты) обслуживания клиентов
                Найти в таблице Coupons у какого офиса наибольшее количество талонов
                """,
"""Найти количество различных талонов в таблице Coupons где месяц = 4 и ВСП имеет значение n
Найти в таблице Coupons среднее количество талонов (полей, т.к. отдельной колонки нет) в офисе n отдельно для понедельника, вторника, среды, четверга, пятницы, субботы и воскресенья
Найти в таблице Coupons среднее время (минуты) ожидания в минутах талонов в офисе n отдельно для понедельника, вторника, среды, четверга, пятницы, субботы и воскресенья
Найти в таблице Coupons в какие часы ожидание талонов в офисе n наибольшее (отсортируй часы по убыванию среднего времени обслуживания)
""",
"""Проверь в таблице Queues, что у офиса n кпэ больше 90
Найти в таблице Employees_busyness среднее время (минуты) простоя по должностям для офиса n
Найти в таблице Employees_busyness среднее время (минуты) активности по должностям для офиса n
Найти должности в таблице Coupons у кого максимальное время (минуты) обслуживания для офиса n
Найти должности в таблице Coupons у кого максимальное время (минуты) ожидания для офиса n
""",
"""Найти среднее время ожидания для каждой категории талонов за апрель для офиса n по таблице Coupons
Найти максимальное (не минимальное!) время ожидания для каждой категории талонов за апрель для офиса n по таблице Coupons
Найти суммарное количество талонов для каждой категории талонов за апрель для офиса n по таблице Coupons
Найти среднее время обслуживания для каждой категории талонов за апрель для офиса n по таблице Coupons
Найти суммарное количество талонов и суммарное количество обслуженных талонов за апрель для офиса по таблице и посчитать процент обслуженных талонов
""",
"""Найти в таблице Queues, что у офиса n значение кпэ
Найти в таблице Employees_busyness среднее время (минуты) простоя по должностям для офиса n
Найти в таблице Employees_busyness среднее время (минуты) активности по должностям для офиса n
Найти должности в таблице Coupons у кого максимальное время (минуты) обслуживания для офиса n
Найти должности в таблице Coupons у кого максимальное время (минуты) ожидания для офиса n
Найти количество различных талонов в таблице Coupons где месяц = 4 и ВСП имеет значение n
Найти в таблице Coupons среднее время (минуты) ожидания в минутах талонов в офисе n отдельно для понедельника, вторника, среды, четверга, пятницы, субботы и воскресенья
Найти в таблице Coupons в какие часы ожидание талонов в офисе n наибольшее (отсортируй часы по убыванию среднего времени обслуживания)
"""
]

In [4]:
test_dict = {"task": tasks, "instruction": instructions}
test_df = pd.DataFrame.from_dict(test_dict)
test_df.to_excel("assistant_instructions_test.xlsx", index=False)

In [5]:
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_community.docstore.in_memory import InMemoryDocstore
from uuid import uuid4
import faiss
from langchain_community.embeddings import GigaChatEmbeddings
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='./backend/.env')
giga_key = os.getenv("GIGA_API_KEY")

giga_embeddings = GigaChatEmbeddings(
    credentials=giga_key,
    verify_ssl_certs=False,
    scope='GIGACHAT_API_CORP'
)


documents = []
for question, instruction in zip(test_df["task"], test_df["instruction"]):
      document = Document(page_content=question, 
                          metadata={"instruction": instruction})
      documents.append(document)
print("Метаданные добавлены")

#создание индекса
index = faiss.IndexFlat(len(giga_embeddings.embed_query("слово")))

#создание векторной базы знаний
vector_store = FAISS(embedding_function=giga_embeddings, index=index, docstore=InMemoryDocstore(), index_to_docstore_id={})
print("База знаний инициализирована")

#индексирование документов
uuids = [str(uuid4()) for _ in range(len(documents))]
print("Документы проиндексированы")

vector_store.add_documents(documents=documents, ids=uuids)
print("База знаний создана")

vector_store.save_local("./backend/question_instructions_test")

C:\Users\Huawei\AppData\Local\Temp\ipykernel_10088\789326996.py:13: LangChainDeprecationWarning: The class `GigaChatEmbeddings` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChatEmbeddings``.
  giga_embeddings = GigaChatEmbeddings(


Метаданные добавлены
База знаний инициализирована
Документы проиндексированы
База знаний создана


In [6]:
vector_store = FAISS.load_local("./backend/question_instructions_test", giga_embeddings, allow_dangerous_deserialization=True)

In [7]:
question = "Как можно оптимизировать работу офиса 026 чтобы сократить время ожидания клиентов"

In [8]:
result = vector_store.similarity_search(question, k=1)[0]
print(result)
result = result.metadata["instruction"]
result

page_content='Как можно оптимизировать работу офиса n чтобы сократить время ожидания клиентов' metadata={'instruction': 'Найти в таблице Queues, что у офиса n значение кпэ\nНайти в таблице Employees_busyness суммарное время (минуты) простоя по должностям для офиса n\nНайти в таблице Employees_busyness суммарное время (минуты) активности по должностям для офиса n\nНайти должности в таблице Coupons у кого максимальное время (минуты) обслуживания для офиса n\nНайти должности в таблице Coupons у кого максимальное время (минуты) ожидания для офиса n\nНайти количество различных талонов в таблице Coupons где месяц = 4 и ВСП имеет значение n\nНайти в таблице Coupons среднее время (минуты) ожидания в минутах талонов в офисе n отдельно для понедельника, вторника, среды, четверга, пятницы, субботы и воскресенья\nНайти в таблице Coupons в какие часы ожидание талонов в офисе n наибольшее (отсортируй часы по убыванию среднего времени обслуживания)\n'}


'Найти в таблице Queues, что у офиса n значение кпэ\nНайти в таблице Employees_busyness суммарное время (минуты) простоя по должностям для офиса n\nНайти в таблице Employees_busyness суммарное время (минуты) активности по должностям для офиса n\nНайти должности в таблице Coupons у кого максимальное время (минуты) обслуживания для офиса n\nНайти должности в таблице Coupons у кого максимальное время (минуты) ожидания для офиса n\nНайти количество различных талонов в таблице Coupons где месяц = 4 и ВСП имеет значение n\nНайти в таблице Coupons среднее время (минуты) ожидания в минутах талонов в офисе n отдельно для понедельника, вторника, среды, четверга, пятницы, субботы и воскресенья\nНайти в таблице Coupons в какие часы ожидание талонов в офисе n наибольшее (отсортируй часы по убыванию среднего времени обслуживания)\n'